In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

In [2]:
### From scratch

class FraudDetectionCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(FraudDetectionCNN, self).__init__()
        
        # First convolutional block
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Second convolutional block
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Third convolutional block
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Global average pooling instead of large FC layers
        self.gap = nn.AdaptiveAvgPool2d(1)
        
        # Smaller fully connected layers
        self.fc1 = nn.Linear(64, 128)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        # Apply convolutional blocks
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        
        # Global average pooling
        x = self.gap(x)
        x = x.view(-1, 64)
        
        # Apply fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.fc2(x)
        
        return x

In [3]:
# Step 1: Set up transformations for the images

train_transform= transforms.Compose([
    transforms.RandomRotation(5),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to fit model input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Standard ImageNet normalization
])


# Step 2: Load the dataset using ImageFolder
train_dataset = ImageFolder(root="/scratch/ihmehta/DISML Project/dataset/train", transform=train_transform)
test_dataset = ImageFolder(root="/scratch/ihmehta/DISML Project/dataset/test", transform=test_transform)
out_of_sample_dataset = ImageFolder(root="/scratch/ihmehta/DISML Project/dataset/out_of_sample_test", transform=test_transform)

# Creating vaidation set
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

# Create DataLoader for batching and shuffling
train_loader = DataLoader(train_subset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
out_of_sample_loader = DataLoader(out_of_sample_dataset, batch_size=16, shuffle=False)

# Create your custom CNN model
model = FraudDetectionCNN(num_classes=3)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

FraudDetectionCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (gap): AdaptiveAvgPool2d(output_size=1)
  (fc1): Linear(in_features=64, out_features=128, bias=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=128, out_features=3, bias=True)
)

In [4]:
# Step 4: Define the loss function and optimizer
# Calculate class weights inverse to their frequency

# Get all training labels
all_labels = []
for _, labels in train_loader:
    all_labels.extend(labels.cpu().numpy())

# Compute weights
class_weights = compute_class_weight('balanced', classes=np.unique(all_labels), y=all_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1.1156846564719638e-05, weight_decay=0.00019961245924845084)

# Create a scheduler to reduce learning rate when validation loss plateaus
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.6902113588053298, patience=3, verbose=True
)

# Initialize lists to track metrics
train_losses = []
train_accuracies = []
val_accuracies = []
best_val_accuracy = 0.0

# Step 5: Train the model with validation after each epoch
num_epochs = 60

def evaluate_model(model, data_loader, device, class_names):
    """
    Evaluate model performance with detailed metrics
    """
    model.eval()
    
    all_preds = []
    all_labels = []
    correct = 0
    total = 0
    
    class_correct = [0] * len(class_names)
    class_total = [0] * len(class_names)
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            total += labels.size(0)
            correct += (preds == labels).sum().item()
            
            # For per-class accuracy
            for i in range(len(preds)):
                label = labels[i].item()
                class_correct[label] += (preds[i] == labels[i]).item()
                class_total[label] += 1
            
            # For confusion matrix and classification report
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    # Calculate overall accuracy
    accuracy = correct / total
    
    # Calculate per-class accuracy
    per_class_accuracy = {}
    for i in range(len(class_names)):
        if class_total[i] > 0:
            per_class_accuracy[class_names[i]] = class_correct[i] / class_total[i]
        else:
            per_class_accuracy[class_names[i]] = 0
    
    # Generate confusion matrix and classification report
    conf_matrix = confusion_matrix(all_labels, all_preds)
    class_report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True,  zero_division=0)
    
    return accuracy, per_class_accuracy, conf_matrix, class_report, all_labels, all_preds


# Get class names from your dataset
class_names = train_dataset.classes  # Should be ['0', '1', '2']

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Track the loss and accuracy
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct_preds += (preds == labels).sum().item()
        total_preds += labels.size(0)
    
    # Calculate epoch accuracy and loss
    epoch_accuracy = correct_preds / total_preds
    epoch_loss = running_loss / len(train_loader)
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)
    
    # Evaluation on validation set (test set in this case)
    val_accuracy, val_per_class_acc, _, _, _, _ = evaluate_model(model, val_loader, device, class_names)
    val_accuracies.append(val_accuracy)
    
    # Update learning rate based on validation performance
    scheduler.step(epoch_loss)
    
    # Save best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), "best_fraud_detection_model.pth")
        print(f"New best model saved with validation accuracy: {best_val_accuracy:.4f}")
    
    # Print epoch statistics
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Loss: {epoch_loss:.4f}, "
          f"Train Accuracy: {epoch_accuracy:.4f}, "
          f"Validation Accuracy: {val_accuracy:.4f}")
    
    # Print per-class accuracy on validation set
    # print("Test Per-class Accuracy:")
    # for cls_name, acc in val_per_class_acc.items():
    #     print(f"  Class {cls_name}: {acc:.4f}")

# Plot training progress
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train')
plt.plot(val_accuracies, label='Validation')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.savefig('training_progress.png')
plt.close()

# Load the best model for final evaluation
model.load_state_dict(torch.load("best_fraud_detection_model.pth"))

# Final evaluation on all datasets
print("\n" + "="*50)
print("FINAL MODEL EVALUATION")
print("="*50)

# Evaluate on training set
print("\nTraining Set Performance:")
train_acc, train_per_class_acc, train_cm, train_report, _, _ = evaluate_model(
    model, train_loader, device, class_names
)
print(f"Overall Accuracy: {train_acc:.4f}")
print("Per-class Accuracy:")
for cls_name, acc in train_per_class_acc.items():
    print(f"  Class {cls_name}: {acc:.4f}")

# Evaluate on test set
print("\nTest Set Performance:")
test_acc, test_per_class_acc, test_cm, test_report, _, _ = evaluate_model(
    model, test_loader, device, class_names
)
print(f"Overall Accuracy: {test_acc:.4f}")
print("Per-class Accuracy:")
for cls_name, acc in test_per_class_acc.items():
    print(f"  Class {cls_name}: {acc:.4f}")

# Evaluate on out-of-sample set
print("\nOut-of-Sample Set Performance:")
oos_acc, oos_per_class_acc, oos_cm, oos_report, _, _ = evaluate_model(
    model, out_of_sample_loader, device, class_names
)
print(f"Overall Accuracy: {oos_acc:.4f}")
print("Per-class Accuracy:")
for cls_name, acc in oos_per_class_acc.items():
    print(f"  Class {cls_name}: {acc:.4f}")

# Create a DataFrame for all metrics to easily compare
metrics_df = pd.DataFrame({
    'Dataset': ['Training', 'Test', 'Out-of-Sample'],
    'Overall Accuracy': [train_acc, test_acc, oos_acc],
    'Class 0 Accuracy': [train_per_class_acc.get('0', 0), test_per_class_acc.get('0', 0), oos_per_class_acc.get('0', 0)],
    'Class 1 Accuracy': [train_per_class_acc.get('1', 0), test_per_class_acc.get('1', 0), oos_per_class_acc.get('1', 0)],
    'Class 2 Accuracy': [train_per_class_acc.get('2', 0), test_per_class_acc.get('2', 0), oos_per_class_acc.get('2', 0)],
    'Class 0 F1-Score': [train_report['0']['f1-score'], test_report['0']['f1-score'], oos_report['0']['f1-score']],
    'Class 1 F1-Score': [train_report['1']['f1-score'], test_report['1']['f1-score'], oos_report['1']['f1-score']],
    'Class 2 F1-Score': [train_report['2']['f1-score'], test_report['2']['f1-score'], oos_report['2']['f1-score']]
})

# Display the metrics table
print("\nComparison of Metrics Across Datasets:")
print(metrics_df.to_string(index=False))

# Save metrics to CSV
metrics_df.to_csv('fraud_detection_metrics.csv', index=False)
print("\nMetrics saved to 'fraud_detection_metrics.csv'")

# Step 7: Save the final model
torch.save(model.state_dict(), "fraud_detection_model.pth")
print("Final model saved as fraud_detection_model.pth")

print("\nModel training and evaluation complete!")

/home/ihmehta/.local/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


New best model saved with validation accuracy: 0.3333
Epoch [1/60], Loss: 1.0994, Train Accuracy: 0.3533, Validation Accuracy: 0.3333
New best model saved with validation accuracy: 0.5238
Epoch [2/60], Loss: 1.0982, Train Accuracy: 0.3904, Validation Accuracy: 0.5238
Epoch [3/60], Loss: 1.0964, Train Accuracy: 0.4675, Validation Accuracy: 0.5119
New best model saved with validation accuracy: 0.6548
Epoch [4/60], Loss: 1.0919, Train Accuracy: 0.4754, Validation Accuracy: 0.6548
Epoch [5/60], Loss: 1.0980, Train Accuracy: 0.5139, Validation Accuracy: 0.5952
New best model saved with validation accuracy: 0.7381
Epoch [6/60], Loss: 1.0946, Train Accuracy: 0.5458, Validation Accuracy: 0.7381
Epoch [7/60], Loss: 1.0997, Train Accuracy: 0.5272, Validation Accuracy: 0.7381
Epoch [8/60], Loss: 1.0949, Train Accuracy: 0.5910, Validation Accuracy: 0.6905
Epoch [9/60], Loss: 1.0938, Train Accuracy: 0.5936, Validation Accuracy: 0.7381
Epoch [10/60], Loss: 1.1003, Train Accuracy: 0.6135, Validation 

/tmp/ipykernel_1919645/3598789031.py:163: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_fraud_detection_model.pth"))



FINAL MODEL EVALUATION

Training Set Performance:
Overall Accuracy: 0.7928
Per-class Accuracy:
  Class 0: 0.9851
  Class 1: 0.0132
  Class 2: 0.0000

Test Set Performance:
Overall Accuracy: 0.8033
Per-class Accuracy:
  Class 0: 1.0000
  Class 1: 0.0000
  Class 2: 0.0000

Out-of-Sample Set Performance:
Overall Accuracy: 0.8000
Per-class Accuracy:
  Class 0: 1.0000
  Class 1: 0.0000
  Class 2: 0.0000

Comparison of Metrics Across Datasets:
      Dataset  Overall Accuracy  Class 0 Accuracy  Class 1 Accuracy  Class 2 Accuracy  Class 0 F1-Score  Class 1 F1-Score  Class 2 F1-Score
     Training          0.792829          0.985124          0.013158               0.0          0.885587          0.022727               0.0
         Test          0.803279          1.000000          0.000000               0.0          0.890909          0.000000               0.0
Out-of-Sample          0.800000          1.000000          0.000000               0.0          0.888889          0.000000               0